# **DATA WRANGLING SITUACIÓN ESTACIONES**
This file attemps to gather, collect, and transform the station situations raw dataset from the source attached below in order to analyse the data avilable and proceed with it. Data collected dates July 2019 to December 2019 (pre-covid). The following processes will be dealt with:

1. Reading the .json files and transforming variables
2. Data exploration
3. Reshaping data
4. Filtering data

<u>Source</u>: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f4b07e0543815610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

<u>Folder</u>: Situación estaciones BiciMAD por día y hora entre julio y diciembre de 2019 

#### **LIBRARIES**

In [1]:
import pandas as pd 
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

#### **1. READ DATA and VARIABLE TRANSFORMATION**
**Dataset**: 20190x.json (X stands for the month number)    

**Description**: Dataset of the situation of the different stations of BiciMAD by date and time

**Dataframe size**: Each month 145,286 timestamps are colected with information on 14 variables. Observations froom july to december total 883,777 timestamps

**Variables**:
- _id_: Código de la Estación Base
- _latitude_: Latitud de la estación en formato WGS84
- _longitude_: Longitud de la Estación en formato WGS84
- _name_: Nombre de la Estación
- _light_: Grado de Ocupación (0=baja, 1=media, 2=alta, 3=inactiva)
- _number_: Denominación lógica de la Estación Base
- _actívate_: Estación activa (0=No activa, 1=activa)
- _no_available_: Disponibilidad de la Estación (0=disponible, 1=no disponible)
- _total_bases_: Número de bases de la estación
- _dock_bikes_: Número de bicicletas ancladas
- _free_bases_: Número de bases libres
- _reservations_count_: Número de reservas activas

In [ ]:
situaciones_list = ["201907", "201908", "201909", "201910", "201911", "201912"]

situaciones = pd.DataFrame()


for i in situaciones_list:
    data = []
    with open('../Data/SituacionEstaciones/'+i+'.json','r') as f:
        for line in f:
            data.append(json.loads(line))

    df_i = pd.json_normalize(
        data, 
        meta=['_id'],
        record_path =['stations']
    )
    # Convert _id to format date
    df_i["_id"] = pd.to_datetime(df_i["_id"])
    # Dorp column id of the base
    situaciones = pd.concat( [situaciones, df_i.drop(columns="id")], ignore_index=True, axis=0)

situaciones.head()

#### **2. DATA EXPLORATION**

In [ ]:
situaciones.shape

In [ ]:
situaciones.to_parquet('../Data/SituacionEstaciones/situaciones.parquet')

In [ ]:
situaciones.head(10)

**Variables type check**: correct

In [ ]:
situaciones.dtypes

**NaN check**: don't exist

In [ ]:
situaciones.isna().sum()

All stations are active. Variable **activate** is irrelevant and can be dropped

In [ ]:
situaciones["activate"].value_counts()

In [ ]:
situaciones.value_counts(["name", "number"])

Distribution of **number of reservations** varies across **stations and months**. 
- December has the lowest number of reservations followed by august. This may beacuase of holidays
- San Hermenegildo is the station that gets more reservations (total=and San Juan the Lowest

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = situaciones["name"],
        y = situaciones.groupby(situaciones["name"])["reservations_count"].sum(),
        name = "Julio"
    )
)
fig.update_layout(title = "Número de reservas total de reservas Jul-Dic 2019 por Estación y Mes", xaxis={'categoryorder':'total descending'})
fig.show()

In [5]:

situaciones=pd.read_parquet('../Data/SituacionEstaciones/situaciones.parquet')
situaciones[['longitude', 'latitude']]=situaciones[['longitude', 'latitude']].apply(pd.to_numeric)
situaciones.loc[:,'weekDay']=situaciones['_id'].apply(lambda x: x.dayofweek)
situaciones.loc[:,'hour']=situaciones['_id'].apply(lambda x: x.hour)
situaciones['number']=situaciones['number'].apply(pd.to_numeric, errors='coerce')
situaciones.to_parquet('../Data/SituacionEstaciones/situaciones2.parquet')

In [7]:

itinerarios_bases=pd.read_parquet('../Data/Itinerarios/itinerarios_bases.parquet')
itinerarios_bases.head()

,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,return_date,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
0,5,1,20,154,44,5,40,2019-07-01 00:00:00+00:00,28007,2019-07-01 02:33:59.999961600+00:00,...,01-02 EMBAJADORES,21.0,40.409817,-3.703914,40,01 CENTRO,01-02 EMBAJADORES,24.0,40.413788,-3.704746
1,8,1,17,184,57,4,129,2019-07-01 00:00:00+00:00,28045,2019-07-01 03:04:00.000019200+00:00,...,01-05 UNIVERSIDAD,24.0,40.429638,-3.712818,129,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438469,-3.698282
2,15,1,16,282,53,4,43,2019-07-01 00:00:00+00:00,nan,2019-07-01 04:41:59.999971200+00:00,...,01-02 EMBAJADORES,24.0,40.408279,-3.700730,43,01 CENTRO,01-02 EMBAJADORES,24.0,40.410088,-3.702545
3,18,1,18,388,56,0,13,2019-07-01 00:00:00+00:00,nan,2019-07-01 06:27:59.999961600+00:00,...,01-06 SOL,24.0,40.418484,-3.703288,13,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427364,-3.710473
4,1,1,20,254,105,0,75,2019-07-01 00:00:00+00:00,nan,2019-07-01 04:14:00.000009600+00:00,...,04-01 RECOLETOS,24.0,40.427067,-3.687474,75,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408340,-3.678415
